In [1]:
from bokeh.plotting import output_notebook
output_notebook()
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
%matplotlib inline

from bokeh.io import curdoc
from bokeh.plotting import figure, show, save, output_file#, ColumnDataSource
from bokeh.models import ColumnDataSource, Slider, TextInput, HoverTool, LinearColorMapper
from bokeh.layouts import widgetbox, row
from bokeh.palettes import brewer

Loading BokehJS ...

In [2]:
# Output inline in the notebook
output_file('Slider.html',
            title='Average sold price around trainstations')

In [3]:
trainlines = gpd.read_file(r'D:\viz\trainlines\viz_layers.gpkg', layer='railway')
clyde = gpd.read_file(r'D:\viz\trainlines\viz_layers.gpkg', layer='river')
#stations  contains schematics geometry and calculated price
stations = gpd.read_file(r'd:\viz\trainlines\derived\New folder\station_price.gpkg', layer='price_dz_points')
station_labels =gpd.read_file(r'D:\viz\trainlines\viz_layers.gpkg', layer='railstation_labels')
stations['mean'] = pd.to_numeric(stations['mean']).round()
station_labels['name'] = station_labels['name'].str.upper()

In [4]:
subway  = gpd.read_file(r'D:\viz\trainlines\viz_layers.gpkg', layer='subway')
subway_labels  = gpd.read_file(r'D:\viz\trainlines\viz_layers.gpkg', layer='subway_lables')
subway_labels['name'] = subway_labels['name'].str.upper()

In [5]:
tsource = ColumnDataSource(trainlines)
stsource = ColumnDataSource(stations)
ssource = ColumnDataSource(subway)
slsource = ColumnDataSource(subway_labels)
stlsource = ColumnDataSource(station_labels)

In [6]:
def getPolyCoords(row, geom, coord_type):
    """Returns the coordinates ('x' or 'y') of edges of a Polygon exterior"""

    # Parse the exterior of the coordinate
    exterior = row[geom].exterior

    if coord_type == 'x':
        # Get the x coordinates of the exterior
        return list( exterior.coords.xy[0] )
    elif coord_type == 'y':
        # Get the y coordinates of the exterior
        return list( exterior.coords.xy[1] )

In [7]:
def getLineCoords(row, geom, coord_type):
    """Returns a list of coordinates ('x' or 'y') of a LineString geometry"""
    if coord_type == 'x':
        return list( row[geom].coords.xy[0] )
    elif coord_type == 'y':
        return list( row[geom].coords.xy[1] )

In [8]:
def getPointCoords(row, geom, coord_type):
    """Calculates coordinates ('x' or 'y') of a Point geometry"""
    if coord_type == 'x':
        return row[geom].x
    elif coord_type == 'y':
        return row[geom].y

In [9]:
# Get the Polygon x and y coordinates
#grid['x'] = grid.apply(getPolyCoords, geom='geometry', coord_type='x', axis=1)
#grid['y'] = grid.apply(getPolyCoords, geom='geometry', coord_type='y', axis=1)

# Calculate x and y coordinates of the line
trainlines['x'] = trainlines.apply(getLineCoords, geom='geometry', coord_type='x', axis=1)
trainlines['y'] = trainlines.apply(getLineCoords, geom='geometry', coord_type='y', axis=1)


clyde['x'] = clyde.apply(getLineCoords, geom='geometry', coord_type='x', axis=1)
clyde['y'] = clyde.apply(getLineCoords, geom='geometry', coord_type='y', axis=1)

In [10]:
# Calculate x and y coordinates of the points
subway['x'] = subway.apply(getPointCoords, geom='geometry', coord_type='x', axis=1)
subway['y'] = subway.apply(getPointCoords, geom='geometry', coord_type='y', axis=1)

subway_labels['x'] = subway_labels.apply(getPointCoords, geom='geometry', coord_type='x', axis=1)
subway_labels['y'] = subway_labels.apply(getPointCoords, geom='geometry', coord_type='y', axis=1)

stations['x'] = stations.apply(getPointCoords, geom='geometry', coord_type='x', axis=1)
stations['y'] = stations.apply(getPointCoords, geom='geometry', coord_type='y', axis=1)

station_labels['x'] = station_labels.apply(getPointCoords, geom='geometry', coord_type='x', axis=1)
station_labels['y'] = station_labels.apply(getPointCoords, geom='geometry', coord_type='y', axis=1)

In [11]:
# Store the data in a ColumnDataSource
#stations['year'] = stations['year'].astype('str')
#stations_s = ColumnDataSource(stations.drop(columns=['geometry','fclass','time_man']))

#stations_s.data.keys()

In [12]:
stations.dtypes

name          object
year           int64
mean         float64
fclass        object
time_man      object
geometry    geometry
x            float64
y            float64
dtype: object

In [13]:
# Calculate x and y coordinates of the points
#subway['x'] = subway.apply(getPointCoords, geom='geometry', coord_type='x', axis=1)
#subway['y'] = subway.apply(getPointCoords, geom='geometry', coord_type='y', axis=1)

stations['x'] = stations.apply(getPointCoords, geom='geometry', coord_type='x', axis=1)
stations['y'] = stations.apply(getPointCoords, geom='geometry', coord_type='y', axis=1)

In [14]:
# Make a copy, drop the geometry column and create ColumnDataSource
tr_df = trainlines.drop('geometry', axis=1).copy()
trsource = ColumnDataSource(tr_df)

# Make a copy, drop the geometry column and create ColumnDataSource
cl_df = clyde.drop('geometry', axis=1).copy()
clsource = ColumnDataSource(cl_df)


# Make a copy, drop the geometry column and create ColumnDataSource
s_df = subway.drop('geometry', axis=1).copy()
ssource = ColumnDataSource(s_df)

# Make a copy, drop the geometry column and create ColumnDataSource
sl_df = subway_labels.drop('geometry', axis=1).copy()
slsource = ColumnDataSource(sl_df)


# Make a copy, drop the geometry column and create ColumnDataSource
st_df = stations.drop('geometry', axis=1).copy()
stsource = ColumnDataSource(st_df)

# Make a copy, drop the geometry column and create ColumnDataSource
stl_df = station_labels.drop('geometry', axis=1).copy()
stlsource = ColumnDataSource(stl_df)

In [15]:
p3 = figure(plot_width=1150, plot_height=800, title='Click Legend to HIDE Data and hover over to see the Average price', title_location = 'above',
          toolbar_location=None)

In [16]:
# Add clyde on top of the same figure
p3.multi_line('x', 'y', source=clsource, color="#5e96cb", line_width=14, line_cap ='round', line_join = 'round')
# Add clyde additional styling 
##TO-DO find a better way to achieve similar styling result
p3.multi_line('x', 'y', source=clsource, color="white", line_width=12, line_cap ='round', line_join = 'round')
p3.multi_line('x', 'y', source=clsource, color="#98bcde", line_width=10, line_cap ='round', line_join = 'round')

GlyphRenderer(id='1060', ...)

In [17]:
# Add trainlines on top of the same figure
p3.multi_line('x', 'y', source=trsource, color="#B0B0B0", line_width=7, line_cap ='round', line_join = 'round')
#just for styling
p3.multi_line('x', 'y', source=trsource, color="white", line_width=5, line_cap ='round', line_join = 'round')
p3.multi_line('x', 'y', source=trsource, color="#B0B0B0", line_width=3, line_cap ='round', line_join = 'round')

GlyphRenderer(id='1077', ...)

In [18]:
# Add subway on top (as black points)
subway = p3.circle('x', 'y', size=10, source=ssource, color="pink", fill_alpha = 0.5)
# Add stations on top (as blue hollow points)

#stations = p3.circle(x="x", y="y",size=1, source=stsource,fill_alpha=0.1,color = '#0e3271')

In [19]:
#colour based on mean value
colormap = LinearColorMapper(palette='Magma256', low=min(stsource.data['mean']), high=max(stsource.data['mean']))

In [35]:
#p.circle(x="x", y="y", source=stsource, color = {'field'  : stsource.data['mean'], 'transform' : colormap})

p3.circle(x="x", y="y", source=stsource,color = {'field': 'mean', 'transform': colormap}, size =10)

GlyphRenderer(id='1314', ...)

In [21]:
# Hover tool referring to our own data field using @ and
# a position on the graph using $
h = HoverTool(tooltips = [('Price ', '@mean'),('Year','@year')])

In [22]:
#show(p3)

In [23]:
# Define the callback function: update_plot
def update_plot(attr, old, new):
    # Set the year name to slider.value and new_data to source.data
    year = slider.value
    stsource.data  = st_df[st_df.year == year]
 

In [24]:
#stations_s = ColumnDataSource(st_df[st_df.year == 2012])


In [25]:
#for key in stations_s.data.keys():
   # for value in stations_s.data[key]:
       # print (key,value, type(value))

In [26]:
# Make a slider object: slider
slider = Slider(title = 'Year', start = 2008, end = 2020, step = 1, value = 2012)

In [27]:
# Hover tool referring to our own data field using @ and
# a position on the graph using $
p3.add_tools(h)

In [28]:
#removing grid lines

p3.xgrid.grid_line_color = None
p3.ygrid.grid_line_color = None

#hiding axis
p3.axis.visible = False

In [29]:
#p3.add_layout(labels1)
#p3.add_layout(labels2)

In [30]:

# Attach the callback to the 'value' property of slider
slider.on_change('value', update_plot)
 
# Make a row layout of widgetbox(slider) and plot and add it to the current document
layout = row(widgetbox(slider), p3)
curdoc().add_root(layout)

In [36]:

show(layout)

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/server.html



You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/server.html



In [32]:
from bokeh.palettes import RdPu9

In [33]:
RdPu9.reverse()

In [34]:
colormap = LinearColorMapper(palette='RdPu9', low=min(stsource.data['mean']), high=max(stsource.data['mean']))